In [ ]:
#!python -c "import torch; print(torch.__version__)"


Found existing installation: pyg-lib 0.1.0+pt113cpu
Uninstalling pyg-lib-0.1.0+pt113cpu:
  Successfully uninstalled pyg-lib-0.1.0+pt113cpu
Found existing installation: torch-scatter 2.1.1+pt113cpu
Uninstalling torch-scatter-2.1.1+pt113cpu:
  Successfully uninstalled torch-scatter-2.1.1+pt113cpu
Found existing installation: torch-sparse 0.6.17+pt113cpu
Uninstalling torch-sparse-0.6.17+pt113cpu:
  Successfully uninstalled torch-sparse-0.6.17+pt113cpu
Found existing installation: torch-cluster 1.6.1+pt113cpu
Uninstalling torch-cluster-1.6.1+pt113cpu:
  Successfully uninstalled torch-cluster-1.6.1+pt113cpu
Found existing installation: torch-spline-conv 1.2.2+pt113cpu
Uninstalling torch-spline-conv-1.2.2+pt113cpu:
  Successfully uninstalled torch-spline-conv-1.2.2+pt113cpu
Found existing installation: torch-geometric 2.2.0
Uninstalling torch-geometric-2.2.0:
  Successfully uninstalled torch-geometric-2.2.0


In [1]:
# dependencies
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
# !pip install torch-geometric
# !pip install pyg-lib -f https://data.pyg.org/whl/torch-2.0.0+cu117.html
# !pip install tables

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu117/torch_scatter-2.1.1%2Bpt20cu117-cp39-cp39-linux_x86_64.whl (10.2 MB)
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu117/torch_sparse-0.6.17%2Bpt20cu117-cp39-cp39-linux_x86_64.whl (4.8 MB)
  Using cached torch_geometric-2.3.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 10.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu117/pyg_lib-0.2.0%2Bpt20cu117-cp39-cp39-linux_x86_64.whl (1.4 MB)


In [1]:
import scipy
import scipy.sparse
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torch.nn import CosineSimilarity

from torch_geometric.data import HeteroData
from torch_geometric.nn import GraphConv, to_hetero
from torch_geometric.loader import NeighborLoader

from tqdm import tqdm
import gc

import random
import glob

In [2]:
# Define correlation loss
def correlation_loss(x1, x2):
    cos = CosineSimilarity(dim=1, eps=1e-6)
    pearson = cos(x1 - x1.mean(dim=1, keepdim=True), x2 - x2.mean(dim=1, keepdim=True))
    return (- 1 * pearson).mean()

In [3]:
# Read input and target data in sparse format
DATA_DIR = "../data/"
train_inputs = scipy.sparse.load_npz(os.path.join(DATA_DIR, "sparse_data/train_multi_inputs_values.sparse.npz"))
target_GEX = scipy.sparse.load_npz(os.path.join(DATA_DIR, "train_multi_targets_values.sparse.npz"))

In [2]:
def create_graph(train_inputs, subset, target_GEX, test_size=2000):
    """
    Prepare and structure a heterogenous graph from given input data.
    """
    
    # Create a new heterogeneous graph data object
    data = HeteroData()
    
    # Initialize node features for 'cell' and 'atac' with tensor of ones
    data['cell'].x = torch.tensor(np.full((len(subset),1),1), dtype=torch.float)
    data['atac'].x = torch.tensor(np.full((228942,1),1), dtype=torch.float)

    # Convert the subset of training inputs to dense format and transpose
    train_inputs_d_t = train_inputs[subset,:].todense().transpose()
    
    # Identify nonzero entries, which indicate the edges, and store them
    tmp = np.nonzero(train_inputs_d_t)
    data['cell','accessibility','atac'].edge_index = torch.tensor([tmp[1].tolist(),tmp[0].tolist()], dtype=torch.long)
    data['atac','rev_accessibility','cell'].edge_index = torch.tensor([tmp[0].tolist(),tmp[1].tolist()], dtype=torch.long)
    
    # Extract edge weights and store them
    tmp_edge_weights = train_inputs_d_t[train_inputs_d_t != 0]
    data['cell','accessibility','atac'].edge_weight = torch.tensor(np.asarray(tmp_edge_weights).flatten(), dtype=torch.float)
    data['atac','rev_accessibility','cell'].edge_weight = torch.tensor(np.asarray(tmp_edge_weights).flatten(), dtype=torch.float)
    
    # Store target values associated with cell nodes
    data['cell'].y = torch.tensor(target_GEX[subset,:].todense(), dtype=torch.float)
    
    # Create masks for training and testing datasets and store them
    train_mask = np.full(len(subset), False)
    train_mask[:(len(subset)-test_size)] = True
    test_mask = np.full(len(subset), False)
    test_mask[(len(subset)-test_size):len(subset)] = True
    data['cell'].train_mask = torch.tensor(train_mask)
    data['cell'].test_mask = torch.tensor(test_mask)
    
    return data
    

In [ ]:
@torch.no_grad() # A decorator that turns off gradient calculation
def init_params():
    """
    Initialize the lazy parameters of the model by forwarding a single batch through it.
    """
    batch = next(iter(train_loader))
    #batch = batch.to(device, 'edge_index')
    model(batch.x_dict, batch.edge_index_dict, batch.edge_weight_dict)

In [2]:
class GNN(torch.nn.Module):
    """
    A GNN with two graph convolutional layers.
    """
    def __init__(self, hidden_channels, out_channels):
        """
        Initializes the GNN model.

        Parameters:
        - hidden_channels: Number of channels in the hidden layer.
        - out_channels: Number of channels in the output layer.
        """
        super().__init__()
        
        # Input channels are lazily determined (-1)
        self.conv1 = GraphConv(-1, hidden_channels, add_self_loops=False, aggr = 'max')
        self.conv2 = GraphConv(-1, out_channels, add_self_loops=False, aggr = 'max')

    def forward(self, x, edge_index, edge_weight):
        """
        Forward pass through the GNN model.

        Parameters:
        - x: Node features.
        - edge_index: Edge indices indicating node pairs.
        - edge_weight: Weights associated with edges.

        Returns:
        - x: Updated node features after passing through the GNN.
        """
        x = self.conv1(x, edge_index, edge_weight=edge_weight).relu()
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        
        return x

In [7]:
def train():
    """
    Train the model on data from train_loader and return the average training loss.
    """
    
    # Set the model to train mode
    model.train()

    total_examples = total_loss = 0
    # Iterate over batches of data from the train_loader
    for batch in tqdm(train_loader):
        
        # Clear the previously calculated gradients
        optimizer.zero_grad()
        
        #batch = batch.to(device, 'edge_index')
        batch_size = batch['cell'].batch_size  
        
        # Forward the batch data through the model
        out = model(batch.x_dict, batch.edge_index_dict, batch.edge_weight_dict)['cell'][:batch_size]
        
        # Calculate the loss
        #loss = F.mse_loss(out, batch['cell'].y[:batch_size])
        loss = correlation_loss(out, batch['cell'].y[:batch_size])
        
        # Compute the gradients through backpropagation
        loss.backward()
        
        # Update the model's parameters based on the gradients
        optimizer.step()

        total_examples += batch_size
        total_loss += float(loss) * batch_size

    return total_loss / total_examples

@torch.no_grad() # A decorator that turns off gradient calculation
def test(loader):
    """
    Evaluate the model's performance on data from the provided loader and returns the average loss on the evaluation data.
    
    Parameters:
        loader: DataLoader object for evaluation data.
    """

    # Set the model to evaluation mode
    model.eval()

    total_examples = total_loss = 0
    # Iterate over batches of data from the loader
    for batch in tqdm(loader):
        #batch = batch.to(device, 'edge_index')
        batch_size = batch['cell'].batch_size  
        out = model(batch.x_dict, batch.edge_index_dict, batch.edge_weight_dict)['cell'][:batch_size]
        #loss = F.mse_loss(out, batch['cell'].y[:batch_size])
        loss = correlation_loss(out, batch['cell'].y[:batch_size])
        total_examples += batch_size
        total_loss += float(loss) * batch_size
        
    return total_loss / total_examples

In [8]:
cell_size = 15000
n_groups = train_inputs.shape[0]//cell_size
numbers = list(range(train_inputs.shape[0]))
random.seed(42)
random.shuffle(numbers)
groups = [numbers[i::n_groups] for i in range(n_groups)]

In [ ]:
data = create_graph(train_inputs, groups[0], target_GEX, test_size=2000)
model = GNN(hidden_channels=64, out_channels=target_GEX.shape[1])
model = to_hetero(model, data.metadata(), aggr='mean')


optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
del data
gc.collect()

In [ ]:
train_loss_dict = {}
test_loss_dict = {}
j = 1

for i in groups:
    print("step0")
    data = create_graph(train_inputs, i, target_GEX, test_size=2000)
    print("step1")
    train_loader = NeighborLoader(data, 
                              num_neighbors=[30] * 2, 
                              input_nodes=('cell', data['cell'].train_mask),
                              batch_size=128)

    test_loader = NeighborLoader(data, 
                                  num_neighbors=[30] * 2, 
                                  input_nodes=('cell', data['cell'].test_mask),
                                  batch_size=128)    
    
    train_loss_dict[j] = {}
    test_loss_dict[j] = {}
    
    for epoch in range(0, 5):
        print(epoch)
        train_loss_dict[j][epoch] = train()
        test_loss_dict[j][epoch] = test(test_loader)
        print(f'Epoch: {epoch:02d}, loss: {train_loss_dict[j][epoch]:.4f}, test_loss: {test_loss_dict[j][epoch]:.4f}')
    print("step2")
    del data
    gc.collect()
    j += 1

In [ ]:
# Save the model's state dictionary
PATH = "../data/heteroGNN_GEX.pt"
torch.save(model.state_dict(), PATH)

In [ ]:
# Create a submision file

In [33]:
# Read data in sparse format
multi_test = scipy.sparse.load_npz(os.path.join(DATA_DIR,"sparse_data/test_multi_inputs_values.sparse.npz"))
multi_test_idxcol = np.load(os.path.join(DATA_DIR,"sparse_data/test_multi_inputs_idxcol.npz", allow_pickle=True))

In [109]:
# This function creates a heterogenous graph for prediction
def create_graph_pred(test_inputs, subset):
    
    data_pred = HeteroData()
    data_pred['cell'].x = torch.tensor(np.full((len(subset),1),1), dtype=torch.float)
    data_pred['atac'].x = torch.tensor(np.full((228942,1),1), dtype=torch.float)
        
    #test_inputs_t = test_inputs.iloc[subset,:].transpose().to_numpy()
    test_inputs_t = test_inputs[subset,:].todense().transpose() # when sparse
    
    tmp = np.nonzero(test_inputs_t)
    data_pred['cell','accessibility','atac'].edge_index = torch.tensor([tmp[1].tolist(),tmp[0].tolist()], dtype=torch.long)
    data_pred['atac','rev_accessibility','cell'].edge_index = torch.tensor([tmp[0].tolist(),tmp[1].tolist()], dtype=torch.long)
    tmp_edge_weights = test_inputs_t[test_inputs_t != 0]

    data_pred['cell','accessibility','atac'].edge_weight = torch.tensor(np.asarray(tmp_edge_weights).flatten(), dtype=torch.float)
    data_pred['atac','rev_accessibility','cell'].edge_weight = torch.tensor(np.asarray(tmp_edge_weights).flatten(), dtype=torch.float)

    return data_pred

In [ ]:
state_dict = torch.load(os.path.join(DATA_DIR,"heteroGNN_GEX.pt"))

In [259]:
batch['cell'].batch_size

57

In [255]:
batch.x_dict['cell']

tensor([[  1., 961.],
        [  1., 962.],
        [  1., 963.],
        ...,
        [  1., 325.],
        [  1., 785.],
        [  1., 721.]])

In [256]:
len(batch.x_dict['cell'])

1017

In [129]:
@torch.no_grad()
def predict(loader, subset):
    model.eval()
    out_pred = {}
    j = 1
    for batch in tqdm(loader):
        out_pred = model(batch.x_dict, batch.edge_index_dict, batch.edge_weight_dict)['cell']
        np.save("../data/batch_multi_pred/batch_multi_prediction_" + str(subset) + "sub_" + str(j),out_pred.detach().numpy())
        j += 1 
        del out_pred
        gc.collect()

In [ ]:
#index_array = np.array(list(zip(multi_test_idxcol_sub, range(1,len(multi_test_idxcol_sub)+1))))
#index_array_v2 = np.column_stack((multi_test_idxcol_sub, range(1,len(multi_test_idxcol_sub)+1)))

In [7]:
cell_size = 15000
n_groups = multi_test.shape[0]//cell_size
numbers = list(range(multi_test.shape[0]))
random.seed(42)
random.shuffle(numbers)
groups_pred = [numbers[i::n_groups] for i in range(n_groups)]
l = 1
#out_pred = {}
for i in groups_pred:
    print("step 0")
    data_pred = create_graph_pred(multi_test, i)
    print("step 1")
    model = GNN(hidden_channels=64, out_channels= 23418)
    model = to_hetero(model, data_pred.metadata(), aggr='mean')
    model.load_state_dict(state_dict)
    print("step 2")
    predict_loader = NeighborLoader(data_pred, 
                                num_neighbors=[30] * 2, 
                                input_nodes=('cell'),
                                batch_size=128)   

    predict(predict_loader, l)
    del data_pred
    del predict_loader
    gc.collect()
    l += 1

step 0
step 1
step 2


100%|██████████| 146/146 [04:13<00:00,  1.74s/it]


step 0
step 1
step 2


100%|██████████| 146/146 [04:13<00:00,  1.73s/it]


step 0
step 1
step 2


100%|██████████| 146/146 [04:14<00:00,  1.74s/it]


In [ ]:
####################

In [2]:
PREDICTION_PATH = '../data/batch_multi_pred/*.npy'
file_list = glob.glob(PREDICTION_PATH)


In [3]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


In [4]:
file_list.sort(key=natural_keys)

In [5]:
# Initialize an empty list to store the data from each file
data_list = []
# Loop through each file and load its data into the list
for file in file_list:
    print(file)
    data = np.load(file)
    data_list.append(data)

# Merge the data from all files into a single NumPy array
merged_data = np.concatenate(data_list)
# Create a data frame using the merged data
df_pred = pd.DataFrame(merged_data)

./data/batch_multi_pred/batch_multi_prediction_1sub_1.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_2.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_3.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_4.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_5.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_6.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_7.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_8.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_9.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_10.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_11.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_12.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_13.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_14.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_15.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_16.npy
./data/batch_multi_pred/batch_multi_prediction_1sub_17.npy
./data

In [6]:
IDX_PREDICTION_PATH = '../data/batch_multi_pred_idx/idx*.npy'
idx_file_list = glob.glob(IDX_PREDICTION_PATH)

In [7]:
idx_file_list = np.sort(idx_file_list)

In [9]:
# Initialize an empty list to store the data from each file
idx_data_list = []
# Loop through each file and load its data into the list
for file in idx_file_list:
    idx_data = np.load(file, allow_pickle=True)
    idx_data_list.append(idx_data)
idx_data_merged = np.concatenate(idx_data_list)

In [10]:
df_pred.index = idx_data_merged

In [11]:
target_GEX_idxcol = np.load("../data/sparse_data/train_multi_targets_idxcol.npz", allow_pickle=True)
df_pred.columns = target_GEX_idxcol['columns']

In [13]:
eval_ids = pd.read_csv("../data/evaluation_ids.csv.zip")

In [14]:
eval_ids_sub = eval_ids[eval_ids['gene_id'].str.contains("^ENSG")]

In [15]:
eval_ids_sub

,row_id,cell_id,gene_id
6812820,6812820,8d287040728a,ENSG00000204091
6812821,6812821,8d287040728a,ENSG00000198938
6812822,6812822,8d287040728a,ENSG00000168495
6812823,6812823,8d287040728a,ENSG00000165527
6812824,6812824,8d287040728a,ENSG00000167414
...,...,...,...
65744175,65744175,2c53aa67933d,ENSG00000134419
65744176,65744176,2c53aa67933d,ENSG00000186862
65744177,65744177,2c53aa67933d,ENSG00000170959
65744178,65744178,2c53aa67933d,ENSG00000107874


In [16]:
del eval_ids

In [17]:
#sum(np.isin(df_pred.index, eval_ids_sub['cell_id']))

In [18]:
eval_ids_sub['tmp_id'] = eval_ids_sub['cell_id'] + "_" + eval_ids_sub['gene_id']

In [19]:
eval_ids_sub


,row_id,cell_id,gene_id,tmp_id
6812820,6812820,8d287040728a,ENSG00000204091,8d287040728a_ENSG00000204091
6812821,6812821,8d287040728a,ENSG00000198938,8d287040728a_ENSG00000198938
6812822,6812822,8d287040728a,ENSG00000168495,8d287040728a_ENSG00000168495
6812823,6812823,8d287040728a,ENSG00000165527,8d287040728a_ENSG00000165527
6812824,6812824,8d287040728a,ENSG00000167414,8d287040728a_ENSG00000167414
...,...,...,...,...
65744175,65744175,2c53aa67933d,ENSG00000134419,2c53aa67933d_ENSG00000134419
65744176,65744176,2c53aa67933d,ENSG00000186862,2c53aa67933d_ENSG00000186862
65744177,65744177,2c53aa67933d,ENSG00000170959,2c53aa67933d_ENSG00000170959
65744178,65744178,2c53aa67933d,ENSG00000107874,2c53aa67933d_ENSG00000107874


In [20]:
len(set(eval_ids_sub['gene_id']))

23418

In [22]:
#df_pred = pd.melt(df_pred.reset_index(), id_vars = 'index')

In [23]:
len(df_pred)

55935

In [24]:
pivot_list = list()
chunk_size = 2000

for i in range(0,len(df_pred),chunk_size):
    print(i)
    row_pivot =pd.melt(df_pred.iloc[i:i+chunk_size].reset_index(),id_vars='index')
    row_pivot['tmp_id'] = row_pivot['index'] + "_" + row_pivot['variable']
    pivot_list.append(row_pivot.merge(eval_ids_sub, on='tmp_id', how='inner')[['row_id', 'value']])



0
2000
4000
6000


IOStream.flush timed out


8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000


In [25]:
import pickle

with open("pivot_list_test", "wb") as fp:   #Pickling
    pickle.dump(pivot_list, fp)

In [26]:
del df_pred
gc.collect()

43

In [27]:
df_pred = pd.concat(pivot_list)

In [30]:
np.save("../data/multi_submision.npy", df_pred.sort_values("row_id"))